In [90]:
import pandas as pd
import numpy as np
import os

In [87]:
df = pd.read_csv("C:/Users/seohe/OneDrive/바탕 화면/Thesis/test2.csv", encoding='cp949')

In [88]:
dataset_cf=df.dropna(axis=0)

In [89]:
dataset_cf

,癤풳ser ID,Venue ID,Venue category ID,Venue category name,Latitude,Longitude,Timezone offset in minutes,UTC time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012
...,...,...,...,...,...,...,...,...
65530,520,4b649083f964a5201dbe2ae3,4bf58dd8d48988d1e4931735,Bowling Alley,40.773181,-73.894966,-240,Sun May 13 22:27:23 +0000 2012
65531,210,4ed1b3bd8b81b473982a1b30,4bf58dd8d48988d121941735,Bar,40.973386,-74.071322,-240,Sun May 13 22:27:49 +0000 2012
65532,1054,4e5590b5814d6ce5cc849483,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),40.870630,-74.097926,-240,Sun May 13 22:28:37 +0000 2012
65533,645,4abe905bf964a5207e8e20e3,4bf58dd8d48988d1d2941735,Sushi Restaurant,40.737601,-74.032009,-240,Sun May 13 22:28:38 +0000 2012


In [91]:
# 파일 생성
dataset_cf.to_csv('./dataset_cf.csv')

평점 부여

In [100]:
import pandas as pd

In [101]:
# 데이터셋 파일 경로
file_path = 'C:/Users/seohe/OneDrive/바탕 화면/thesis/dataset_cf.csv'

In [102]:
# 데이터셋 로드
data = pd.read_csv(file_path)

In [103]:
# 각 장소의 방문 횟수를 계산하여 인기도 기반 평점 부여
venue_count = data['Venue ID'].value_counts()
venue_count_dict = venue_count.to_dict()
data['rating'] = data['Venue ID'].apply(lambda x: venue_count_dict[x])
data.to_csv('C:/Users/seohe/OneDrive/바탕 화면/thesis/dataset_cf.csv', index=True)

In [104]:
# Surprise 데이터셋으로 변환
from surprise import Dataset
from surprise import Reader

In [108]:
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(1, 5))
data = Dataset.load_from_df(data[['癤풳ser ID', 'Venue ID', 'rating']], reader=reader)

샘플링 -> 협업필터링 구현

In [119]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd

In [121]:
# 데이터 로드
df = pd.read_csv('C:/Users/seohe/OneDrive/바탕 화면/thesis/dataset_cf.csv')

In [122]:
# 데이터 형식 지정
reader = Reader(rating_scale=(1, 5))

In [124]:
# 데이터프레임을 데이터셋으로 변환
data = Dataset.load_from_df(df[['User ID', 'Venue ID', 'rating']], reader)

In [125]:
# 학습용 데이터와 테스트용 데이터로 분할
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [126]:
# SVD 모델 학습
model = SVD(n_factors=100, n_epochs=20, random_state=42)
model.fit(trainset)

In [127]:
# 테스트 데이터로 모델 평가
predictions = model.test(testset)
print('RMSE:', accuracy.rmse(predictions))

RMSE: 34.2947
RMSE: 34.294662222495525


In [138]:
from surprise.model_selection import train_test_split
from surprise import Dataset

# 데이터셋 불러오기
data = Dataset.load_builtin('ml-100k')

# 랜덤 샘플링을 통해 학습셋과 검증셋 나누기
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 모델 학습 및 예측
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

# 평가
accuracy.rmse(predictions)

RMSE: 0.9373


0.9373187455429636

정밀도, 재현율, f

In [139]:
from surprise import accuracy
from sklearn.metrics import precision_recall_fscore_support

# 학습 데이터로 모델 학습
model.fit(trainset)

# 테스트 데이터로 모델 평가
predictions = model.test(testset)

# 예측값과 실제값을 리스트로 저장
pred = []
true = []
for uid, iid, r_ui, est, _ in predictions:
    pred.append(est)
    true.append(r_ui)

# NaN 값 제거
pred = np.nan_to_num(pred)

# 정밀도, 재현율, F1-score 계산
precision, recall, f1_score, _ = precision_recall_fscore_support(true, np.around(pred), average='weighted')
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1_score)

Precision:  0.46393780387010286
Recall:  0.41915
F1-score:  0.3802322028418893
